<a href="https://colab.research.google.com/github/graceebc9/Arch_Class/blob/main/Param_sweeps_svi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install wandb
import wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 5.5 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=0b0c14f07f2734762032f6ebf72964de2bc2d8ce7d0f257c4f4263041a618ece
  Stored in directory: /root/.cache/pip/wheels/b7/0a/67/ada2a22079218c75a88361c0782855cc72aebc4d18d0289d05
Successfully built pathtools


In [2]:

!pip install pytorch_lightning
import pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 716.4/716.4 KB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 16.5 MB/s eta 0:00:00


In [3]:
import pytorch_lightning
import wandb

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import torchvision.models as models

In [4]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:

import sys

In [6]:
# To run this, update base_dir to be the location on your G-Drive of the HurricaneDamage_Data folder - https://drive.google.com/drive/folders/1b9qMhMblYRnJHzZOqJeFnltUX9jpfLTo
base_dir = '/content/gdrive/MyDrive/' 

import sys
sys.path.insert(0, base_dir + '/HurricanDamage_Data')

from src import  ImagePredictionLogger, ImagePredictionLogger_table 


In [7]:
!apt-get install git
!git clone https://github.com/graceebc9/Arch_Class.git

base_dir = '/content'
sys.path.insert(0, base_dir + '/Arch_Class')


Reading package lists... Done
Building dependency tree       
Reading state information... Done
git is already the newest version (1:2.25.1-1ubuntu3.10).
The following package was automatically installed and is no longer required:
  libnvidia-common-525
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.
Cloning into 'Arch_Class'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 85 (delta 40), reused 36 (delta 13), pack-reused 0
Unpacking objects: 100% (85/85), 2.24 MiB | 3.66 MiB/s, done.


In [8]:

 from utils import SVI_module, StreetViewData, SVI_Resnet



In [9]:
local_data_dir = "/content/gdrive/MyDrive/Non_Trads_FOI/data/full_svi/all_images"

def param_sweep_train_model():
    """
    Parameter sweep in Wandb for ResNet model  
    """


    wandb.init()
    config = wandb.config
    print(config) 
    LR = wandb.config.lr 
    optimiser = wandb.config.optimiser

    dm = SVI_module(data_dir = local_data_dir , batch_size=wandb.config.batch_size )
    
    

    dm.setup()

    # Samples required by the custom ImagePredictionLogger callback to log image predictions.
    val_samples = next(iter(dm.val_dataloader()))
    test_samples = next(iter(dm.test_dataloader()))

    wandb_logger = WandbLogger(project= wand_proj_name, job_type='svi_full', entity ='grace_ebc' )

    trainer = pl.Trainer(
                    max_epochs=20,
                    # progress_bar_refresh_rate=20, 
                    # gpus=1, 
                    logger=wandb_logger,
                    callbacks=[#ModelCheckpoint(save_weights_only=True, mode="max", monitor="val_acc"),
                                ImagePredictionLogger(val_samples), 
                                # ImagePredictionLogger_table( val_samples, 'val' ),
                              EarlyStopping(monitor="val_loss", mode="min")
                              ]

                    )                                                   


    pl.seed_everything(42) 
    
    
    model = SVI_Resnet(model=models.resnet50(weights='DEFAULT'), learning_rate = LR, optimiser = optimiser, class_names = dm.dataset.classes ) #



    trainer.fit(model, dm)

    # Evaluate the model on the held-out test set
    trainer.test(dataloaders = dm.test_dataloader())

    # Close wandb run
    wandb.finish()

In [10]:
#####User to update variables 
#How many sweeps
num_runs = 40

#Update variables 
wand_proj_name = 'full_svi' #wandb proj name 




#Update if you want to use own data
local_data_dir = "/content/gdrive/MyDrive/Non_Trads_FOI/data/full_svi/all_images"


In [11]:
# wandb.login()

In [12]:

# sweep_configuration = {
#     'method': 'random',
#     'name': 'sweep',
#     'metric': {
#         'goal': 'minimize', 
#         'name': 'val_loss'
#         },
#     'parameters': {
#         'bs': {'values': [16, 32, 64]},
#         'epochs': {'values': [5, 10, 15]},
#         'lr': {'max': 0.1, 'min': 0.0001},
#         'optimiser': {'values': ['Adam', 'Adagrad' ] }
#      }
# }

In [13]:
# sweep_config = {
#     'method': 'random'
#     }

  
# metric = {
#     'name': 'val_jac',
#     'goal': 'maximize'   
#     }

# sweep_config['metric'] = metric


# parameters_dict = {
#     'optimiser': {
#         'values': ['Adam', 'Adagrad' , ] }, #'RMSProp', 'SGD'
#     # 'weights' : { 
#     #     'values' : [ [1,1] , [1/10, 1] , [1/5, 1 ], [1, 1/5 ] , [1/100, 1] ]},  
#        }

# sweep_config['parameters'] = parameters_dict


# parameters_dict.update({
#     'learning_rate': {
#         # a flat distribution between 0 and 0.1
#         'distribution': 'uniform',
#         'min': 0,
#         'max': 0.1
#       },
#     'batch_size': {
#         # integers between 32 and 256
#         # with evenly-distributed logarithms 
#         'distribution': 'q_log_uniform_values',
#         'q': 8,
#         'min': 32,
#         'max': 500, #256
#       }
#     })

In [14]:

# sweep_id = wandb.sweep(sweep_config)

In [15]:
# import pprint

# pprint.pprint(sweep_configuration)

In [16]:

# sweep_id = wandb.sweep(sweep_configuration, project=wand_proj_name, entity = 'grace_ebc')

In [17]:

# wandb.agent(sweep_id, param_sweep_train_model(local_data_dir) )


In [ ]:
import wandb

num_runs = 40

#Update variables 
wand_proj_name = 'full_svi' #wandb proj name 




sweep_configuration = {
 "name": "my-awesome-sweep",
 "metric": {"name": "val_acc", "goal": "minimize"},
 "method": "random",
 "parameters": {"a": {"values": [1, 2, 3, 4]} 
                ,'lr': {'max': 0.1, 'min': 0.0001} 
                ,'batch_size': {'values': [16, 32, 64]} 
                ,'optimiser': {'values': ['Adam', 'Adagrad' ] } },
}







sweep_id = wandb.sweep(sweep_configuration, project=wand_proj_name, entity = 'grace_ebc')

# run the sweep
wandb.agent(sweep_id, function=param_sweep_train_model )

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: sulkatds
Sweep URL: https://wandb.ai/grace_ebc/full_svi/sweeps/sulkatds


wandb: Agent Starting Run: 2f0bulwv with config:
wandb: 	a: 1
wandb: 	batch_size: 16
wandb: 	lr: 0.03697957817718911
wandb: 	optimiser: Adam
wandb: Currently logged in as: grace_ebc. Use `wandb login --relogin` to force relogin


{'a': 1, 'batch_size': 16, 'lr': 0.03697957817718911, 'optimiser': 'Adam'}
['PCC', 'ISC', 'TIM', 'MET']
Num train:6197 Num val:1327 Num test:1329
Total num: 8853  total sum: 8853


...

tensor(0.3750) tensor(0.3125) tensor(0.3182)
tensor(0.4375) tensor(0.3250) tensor(0.3333)
tensor(0.5000) tensor(0.3500) tensor(0.3000)
tensor(0.5625) tensor(0.5357) tensor(0.5417)
tensor(0.5625) tensor(0.4643) tensor(0.5114)
tensor(0.3750) tensor(0.3333) tensor(0.4583)
tensor(0.8125) tensor(0.6000) tensor(0.6923)
tensor(0.9375) tensor(0.9500) tensor(0.9722)
tensor(0.6250) tensor(0.3000) tensor(0.4000)
tensor(0.7500) tensor(0.4792) tensor(0.6042)
tensor(0.7500) tensor(0.7917) tensor(0.7917)
tensor(0.8125) tensor(0.6750) tensor(0.6786)
tensor(0.5625) tensor(0.2738) tensor(0.2917)
tensor(0.7500) tensor(0.6875) tensor(0.9000)
tensor(0.5625) tensor(0.3896) tensor(0.5536)
tensor(0.6250) tensor(0.4762) tensor(0.5139)
tensor(0.5625) tensor(0.5417) tensor(0.7611)
tensor(0.6250) tensor(0.6071) tensor(0.8125)
tensor(0.4375) tensor(0.3409) tensor(0.3182)
tensor(0.6250) tensor(0.5429) tensor(0.5804)
tensor(0.5625) tensor(0.7708) tensor(0.7803)
tensor(0.5625) tensor(0.4595) tensor(0.4345)
tensor(0.6

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Validation: 0it [00:00, ?it/s]

tensor(0.4375) tensor(0.6667) tensor(0.4167)
tensor(0.5625) tensor(0.4107) tensor(0.4313)
tensor(0.5000) tensor(0.5104) tensor(0.3958)
tensor(0.3750) tensor(0.2917) tensor(0.2250)
tensor(0.5000) tensor(0.2778) tensor(0.2857)
tensor(0.7500) tensor(0.3667) tensor(0.4222)
tensor(0.5625) tensor(0.6595) tensor(0.6583)
tensor(0.5625) tensor(0.6736) tensor(0.6375)
tensor(0.2500) tensor(0.2500) tensor(0.1778)
tensor(0.4375) tensor(0.2857) tensor(0.3864)
tensor(0.5625) tensor(0.6083) tensor(0.4792)
tensor(0.6875) tensor(0.4821) tensor(0.5694)
tensor(0.7500) tensor(0.8750) tensor(0.8889)
tensor(0.4375) tensor(0.2833) tensor(0.2750)
tensor(0.5625) tensor(0.3056) tensor(0.3214)
tensor(0.3750) tensor(0.1833) tensor(0.2917)
tensor(0.6875) tensor(0.6250) tensor(0.5568)
tensor(0.7500) tensor(0.8188) tensor(0.8417)
tensor(0.4375) tensor(0.3500) tensor(0.2604)
tensor(0.5625) tensor(0.5500) tensor(0.6667)
tensor(0.6250) tensor(0.4875) tensor(0.5833)
tensor(0.7500) tensor(0.6012) tensor(0.6458)
tensor(0.6

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
